In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

In [ ]:
df = pd.read_excel('telcom_data.xlsx')
df.head(5)

In [ ]:
df.columns

In [ ]:
sessions_per_user = df.groupby('MSISDN/Number').size().reset_index(name='xDR Session Count')

In [ ]:
sessions_per_user

Total Session Duration Per Hour


In [ ]:
session_duration = df.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index(name='Total Duration (ms)')
session_duration

In [ ]:
dl_ul_total = df.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()
dl_ul_total

In [ ]:
# Identify application columns
app_columns = [col for col in df.columns if any(app in col for app in [
    'Youtube', 'Netflix', 'Gaming', 'Google', 'Email', 'Social Media', 'Other'
])]

# Aggregate per user
app_usage = df.groupby('MSISDN/Number')[app_columns].sum().reset_index()


In [ ]:
# Merge all summaries step by step
result = sessions_per_user \
    .merge(session_duration, on='MSISDN/Number') \
    .merge(dl_ul_total, on='MSISDN/Number') \
    .merge(app_usage, on='MSISDN/Number')


In [ ]:
print(result.head())  # Shows user-wise summary


In [ ]:
df.fillna(df.mean(numeric_only=True), inplace=True)

In [ ]:
#Outlier Treatment

In [ ]:
for col in df.select_dtypes(include=np.number).columns:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = np.clip(df[col], lower, upper)

In [ ]:
#Basic Metrics (mean, median, std, etc.)

In [ ]:
basic_stats = df.describe().T[['mean', '50%', 'std', 'min', 'max']]
basic_stats.rename(columns={'50%': 'median'}, inplace=True)
print(basic_stats)


In [ ]:
# Non-Graphical Univariate Analysis (Dispersion Parameters)

In [ ]:
dispersion = df.select_dtypes(include=np.number).agg(['mean', 'std', 'var', 'min', 'max']).T
print(dispersion)

In [ ]:
# Graphical Univariate Analysis

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Bivariate Analysis – App vs Total Data (DL + UL)

In [ ]:
df['Total_Data(Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

In [ ]:
app_cols = [col for col in df.columns if 'DL (Bytes)' in col or 'UL (Bytes)' in col and any(x in col for x in ['Youtube', 'Netflix', 'Email', 'Google', 'Social Media', 'Gaming', 'Other'])]

for col in app_cols:
    if col != 'Total_Data(Bytes)':
        sns.scatterplot(x=df[col], y=df['Total_Data(Bytes)'])
        plt.title(f"{col} vs Total Data")
        plt.show()

In [ ]:
#Variable Transformation – User Segmentation by Decile of Duration

In [ ]:
df['Decile'] = pd.qcut(df['Dur. (ms)'], 10, labels=False, duplicates='drop')

# Now compute data usage per decile
decile_analysis = df.groupby('Decile')['Total_Data(Bytes)'].sum().reset_index()
print(decile_analysis)


In [ ]:
#Correlation Analysis – Application Data

In [ ]:
app_related = [col for col in df.columns if any(x in col for x in ['Youtube', 'Netflix', 'Email', 'Google', 'Social Media', 'Gaming', 'Other'])]

correlation_matrix = df[app_related].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix – Application Data")
plt.show()


In [ ]:
#Principal Component Analysis (PCA)

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df[app_related].fillna(0)
X_scaled = StandardScaler().fit_transform(X)

pca = PCA()
pca_components = pca.fit_transform(X_scaled)

explained_variance = pca.explained_variance_ratio_
print(explained_variance)



In [ ]:
plt.figure(figsize=(6,4))
plt.plot(np.cumsum(explained_variance))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA - Explained Variance')
plt.grid()
plt.show()


In [ ]:
# Aggregate Engagement Metrics per User

In [ ]:
# Step 1 - Aggregate required metrics per user
df['Total_Traffic(Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

user_metrics = df.groupby('MSISDN/Number').agg({
    'Dur. (ms)': 'sum',
    'Total_Traffic(Bytes)': 'sum',
    'MSISDN/Number': 'count'  # session count = frequency
}).rename(columns={'MSISDN/Number': 'Session Frequency',
                   'Dur. (ms)': 'Total Duration',
                   'Total_Traffic(Bytes)': 'Total Traffic'}).reset_index()


In [ ]:
#Report Top 10 Users per Metric

In [ ]:
# Top 10 by frequency
print(user_metrics.nlargest(10, 'Session Frequency'))

# Top 10 by duration
print(user_metrics.nlargest(10, 'Total Duration'))

# Top 10 by traffic
print(user_metrics.nlargest(10, 'Total Traffic'))


In [ ]:
#Normalize Metrics & Apply K-Means (k=3)
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Normalize
scaler = MinMaxScaler()
engagement_scaled = scaler.fit_transform(user_metrics[['Session Frequency', 'Total Duration', 'Total Traffic']])

# Apply KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
user_metrics['Engagement Cluster'] = kmeans.fit_predict(engagement_scaled)


In [ ]:
#Cluster-wise Stats (non-normalized)

In [ ]:
cluster_stats = user_metrics.groupby('Engagement Cluster')[['Session Frequency', 'Total Duration', 'Total Traffic']].agg(['min', 'max', 'mean', 'sum'])
print(cluster_stats)

In [ ]:
#Total Traffic Per Application Per User

In [ ]:
app_cols = [col for col in df.columns if any(x in col for x in [
    'Youtube', 'Netflix', 'Email', 'Google', 'Social Media', 'Gaming', 'Other'
])]

# Add total app traffic columns
df['MSISDN/Number'] = df['MSISDN/Number'].astype(str)
app_traffic = df.groupby('MSISDN/Number')[app_cols].sum().reset_index()


In [ ]:
#Top 10 Most Engaged Users Per App

In [ ]:
top_users_per_app = {}
for col in app_cols:
    top_users_per_app[col] = app_traffic[['MSISDN/Number', col]].nlargest(10, col)


In [ ]:
#Top 3 Most Used Applications – Visualize

In [ ]:
app_total = df[app_cols].sum().sort_values(ascending=False).head(3)

# Plot
app_total.plot(kind='bar', title='Top 3 Most Used Applications', ylabel='Total Bytes')
plt.tight_layout()
plt.show()


In [ ]:
# Optimize K Using Elbow Method

In [ ]:
# Elbow Method
inertia = []
K = range(1, 11)

for k in K:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(engagement_scaled)
    inertia.append(km.inertia_)

# Plot elbow
plt.plot(K, inertia, marker='o')
plt.title("Elbow Method - Optimal k")
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
numeric_cols = [
    'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]

for col in numeric_cols:
    df[col].fillna(df[col].mean(), inplace=True)

df['Handset Type'].fillna(df['Handset Type'].mode()[0], inplace=True)

# Treat outliers (clip to 1st and 99th percentile)
for col in numeric_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = np.clip(df[col], lower, upper)

# Create aggregate experience metrics
df['Avg_TCP_Retrans'] = (df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']) / 2
df['Avg_RTT'] = (df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']) / 2
df['Avg_Throughput'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2

# Aggregate per user
user_experience = df.groupby('MSISDN/Number').agg({
    'Avg_TCP_Retrans': 'mean',
    'Avg_RTT': 'mean',
    'Avg_Throughput': 'mean',
    'Handset Type': lambda x: x.mode()[0]  # most frequent handset type
}).reset_index()

print(user_experience.head())

In [ ]:
#Top, Bottom, and Most Frequent Values

In [ ]:
# a. TCP values
tcp_values = df['Avg_TCP_Retrans']
print("Top 10 TCP retransmission values:\n", tcp_values.nlargest(10))
print("Bottom 10 TCP retransmission values:\n", tcp_values.nsmallest(10))
print("Most frequent TCP retransmission values:\n", tcp_values.value_counts().head(10))

# b. RTT values
rtt_values = df['Avg_RTT']
print("Top 10 RTT values:\n", rtt_values.nlargest(10))
print("Bottom 10 RTT values:\n", rtt_values.nsmallest(10))
print("Most frequent RTT values:\n", rtt_values.value_counts().head(10))

# c. Throughput values
tp_values = df['Avg_Throughput']
print("Top 10 Throughput values:\n", tp_values.nlargest(10))
print("Bottom 10 Throughput values:\n", tp_values.nsmallest(10))
print("Most frequent Throughput values:\n", tp_values.value_counts().head(10))


In [ ]:
#Distribution by Handset Type

In [ ]:
throughput_by_handset = user_experience.groupby('Handset Type')['Avg_Throughput'].mean().sort_values(ascending=False)
print(throughput_by_handset)

# Plot
plt.figure(figsize=(10, 6))
throughput_by_handset.head(10).plot(kind='bar')
plt.title("Avg Throughput per Handset Type (Top 10)")
plt.ylabel("Throughput (kbps)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
##TCP retransmission per Handset Type

In [ ]:
tcp_by_handset = user_experience.groupby('Handset Type')['Avg_TCP_Retrans'].mean().sort_values(ascending=False)
print(tcp_by_handset)

plt.figure(figsize=(10, 6))
tcp_by_handset.head(10).plot(kind='bar', color='orange')
plt.title("Avg TCP Retransmission per Handset Type (Top 10)")
plt.ylabel("TCP Retransmission (Bytes)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#KMeans Clustering on Experience Metrics

In [ ]:
experience_features = user_experience[['Avg_TCP_Retrans', 'Avg_RTT', 'Avg_Throughput']]
scaler = MinMaxScaler()
experience_scaled = scaler.fit_transform(experience_features)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
user_experience['Experience Cluster'] = kmeans.fit_predict(experience_scaled)

# Cluster Summary
cluster_summary = user_experience.groupby('Experience Cluster')[['Avg_TCP_Retrans', 'Avg_RTT', 'Avg_Throughput']].mean()
print(cluster_summary)

# Visualizing cluster distribution
sns.pairplot(user_experience, hue='Experience Cluster', vars=['Avg_TCP_Retrans', 'Avg_RTT', 'Avg_Throughput'])
plt.suptitle("User Experience Clusters", y=1.02)
plt.show()


In [ ]:
#Engagement & Experience Scores (Euclidean Distance)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

# Engagement features and clustering (from Task 2)
engagement_features = ['Session Frequency', 'Total Duration', 'Total Traffic']
engagement_data = user_metrics[engagement_features]
least_engaged_center = engagement_data[user_metrics['Engagement Cluster'] == user_metrics['Engagement Cluster'].value_counts().idxmin()].mean().values

# Experience features and clustering (from Task 3)
experience_features = ['Avg_TCP_Retrans', 'Avg_RTT', 'Avg_Throughput']
experience_data = user_experience[experience_features]
worst_experience_center = experience_data[user_experience['Experience Cluster'] == user_experience['Experience Cluster'].value_counts().idxmax()].mean().values

# Compute distances (scores)
user_metrics['Engagement Score'] = euclidean_distances(engagement_data, [least_engaged_center]).flatten()
user_experience['Experience Score'] = euclidean_distances(experience_data, [worst_experience_center]).flatten()


In [ ]:
#Satisfaction Score & Top 10 Customers

In [ ]:
# Ensure MSISDN is string in both DataFrames
user_metrics['MSISDN/Number'] = user_metrics['MSISDN/Number'].astype(str)
user_experience['MSISDN/Number'] = user_experience['MSISDN/Number'].astype(str)

# Now safe to merge
final_df = pd.merge(
    user_metrics[['MSISDN/Number', 'Engagement Score']],
    user_experience[['MSISDN/Number', 'Experience Score']],
    on='MSISDN/Number'
)



In [ ]:
final_df['Satisfaction Score'] = final_df[['Engagement Score', 'Experience Score']].mean(axis=1)

# Top 10 satisfied users
top_satisfied = final_df.sort_values(by='Satisfaction Score', ascending=False).head(10)
print(top_satisfied)

In [ ]:
#Regression Model to Predict Satisfaction

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Use engagement and experience scores to predict satisfaction
X = final_df[['Engagement Score', 'Experience Score']]
y = final_df['Satisfaction Score']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))


In [ ]:
#KMeans on Engagement + Experience Scores

In [ ]:
from sklearn.cluster import KMeans

kmeans_satisfaction = KMeans(n_clusters=2, random_state=42)
final_df['Satisfaction Cluster'] = kmeans_satisfaction.fit_predict(final_df[['Engagement Score', 'Experience Score']])


In [ ]:
# Cluster Aggregation of Scores

In [ ]:
cluster_avg = final_df.groupby('Satisfaction Cluster')[['Satisfaction Score', 'Experience Score']].mean()
print(cluster_avg)


In [ ]:
#Export to MySQL

In [ ]:
import pyodbc

conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=DESKTOP-SFKESGV\SQLEXPRESS;'  # Change to your server
    r'DATABASE=telecom_data;'             # Change to your DB name
    r'Trusted_Connection=yes;'
)


cursor = conn.cursor()
cursor.execute("""
IF OBJECT_ID('dbo.satisfaction_analysis', 'U') IS NULL
BEGIN
    CREATE TABLE dbo.satisfaction_analysis (
        msisdn VARCHAR(255) PRIMARY KEY,
        engagement_score FLOAT,
        experience_score FLOAT,
        satisfaction_score FLOAT,
        satisfaction_cluster INT
    )
END
""")


In [ ]:


for _, row in final_df.iterrows():
    cursor.execute("""
        MERGE INTO dbo.satisfaction_analysis AS target
        USING (SELECT ? AS msisdn) AS source
        ON target.msisdn = source.msisdn
        WHEN MATCHED THEN
            UPDATE SET engagement_score = ?, experience_score = ?, satisfaction_score = ?, satisfaction_cluster = ?
        WHEN NOT MATCHED THEN
            INSERT (msisdn, engagement_score, experience_score, satisfaction_score, satisfaction_cluster)
            VALUES (?, ?, ?, ?, ?);
    """, (
        row['MSISDN/Number'],
        row['Engagement Score'], row['Experience Score'], row['Satisfaction Score'], row['Satisfaction Cluster'],
        row['MSISDN/Number'], row['Engagement Score'], row['Experience Score'], row['Satisfaction Score'], row['Satisfaction Cluster']
    ))

conn.commit()

In [ ]:
cursor.close()
conn.close()

In [ ]:
#Model Deployment & Tracking (Lightweight MLOps)

In [ ]:
import mlflow
import mlflow.sklearn
import time

mlflow.set_experiment("satisfaction_model_tracking")

with mlflow.start_run():
    start_time = time.time()

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("train_size", len(X_train))
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2_score", r2)
    
    mlflow.sklearn.log_model(model, "model")

    end_time = time.time()
    mlflow.log_metric("training_duration", end_time - start_time)
